# Entity Framework Core Examples

Making use of notebooks to printout sql queries formed by EF.

Issues:
- "using var ctx..." gives error when "using" is used

In [21]:
#r "nuget: Microsoft.EntityFrameworkCore.SqlServer, 7.0.5"

using Microsoft.EntityFrameworkCore;
using System;

public class ItemContext : DbContext
{
	public DbSet<Item> Items { get; set; }
	public DbSet<Tag> Tags { get; set; }

	protected override void OnConfiguring(DbContextOptionsBuilder optionsBuilder)
	{
		optionsBuilder.UseSqlServer("Server=(localdb)\\MSSQLLocalDB;Database=SomeDbName;Trusted_Connection=True");
	}
}

public class Item
{
	public int Id { get; set; }
	public string Name { get; set; } = string.Empty;
	public string Status { get; set; } = string.Empty;
	public int SomeId { get; set; }
	public DateTimeOffset Created { get; set; }
	public string EnumValue { get; set; } = string.Empty;
}

public class Tag
{
	public int Id { get; set; }
	public string Name { get; set; } = string.Empty;
	public string Value { get; set; } = string.Empty;
}

Installed Packages Microsoft.EntityFrameworkCore.SqlServer, 7.0.5

### SELECT queries

In [22]:
var ctx = new ItemContext();
IQueryable<Item> query = ctx.Items;

// as long as we do not call ToList or FirstOrDefault
// the following LINQ will not attempt to establish
// connection to database defined in connection string

var itemId = 10;
query = ctx.Items.Where(item => item.Id > itemId);
Console.WriteLine(query.ToQueryString());

DECLARE @__itemId_0 int = 10;

SELECT [i].[Id], [i].[Created], [i].[EnumValue], [i].[Name], [i].[SomeId], [i].[Status]
FROM [Items] AS [i]
WHERE [i].[Id] > @__itemId_0


In [23]:
// when we use raw values in the expression, we do not get parameterized queries
// compare with the following:
query = ctx.Items.Where(item => item.Id > 10);
Console.WriteLine(query.ToQueryString());

SELECT [i].[Id], [i].[Created], [i].[EnumValue], [i].[Name], [i].[SomeId], [i].[Status]
FROM [Items] AS [i]
WHERE [i].[Id] > 10


In [24]:
var nameToMatch = "HELLO WORLD";
query = ctx.Items.Where(item => item.Name.ToUpper().Contains(nameToMatch));
Console.WriteLine(query.ToQueryString());

DECLARE @__nameToMatch_0 nvarchar(4000) = N'HELLO WORLD';

SELECT [i].[Id], [i].[Created], [i].[EnumValue], [i].[Name], [i].[SomeId], [i].[Status]
FROM [Items] AS [i]
WHERE (@__nameToMatch_0 LIKE N'') OR CHARINDEX(@__nameToMatch_0, UPPER([i].[Name])) > 0


In [25]:
query = ctx.Items.Where(item => item.Created < DateTime.Now && item.Id == itemId);
Console.WriteLine(query.ToQueryString());

DECLARE @___itemId_0 int = 10;

SELECT [i].[Id], [i].[Created], [i].[EnumValue], [i].[Name], [i].[SomeId], [i].[Status]
FROM [Items] AS [i]
WHERE [i].[Created] < CAST(GETDATE() AS datetimeoffset) AND [i].[Id] = @___itemId_0


In [26]:
// break up Where clauses -> doesn't seem to affect SQL generated
query = ctx.Items.Where(item => item.Created < DateTime.Now).Where(item => item.Id == 300);
Console.WriteLine(query.ToQueryString());

SELECT [i].[Id], [i].[Created], [i].[EnumValue], [i].[Name], [i].[SomeId], [i].[Status]
FROM [Items] AS [i]
WHERE [i].[Created] < CAST(GETDATE() AS datetimeoffset) AND [i].[Id] = 300
